# Setup

First, let's import a few common modules, ensure MatplotLib plots figures inline and prepare a function to save the figures. We also check that Python 3.5 or later is installed (although Python 2.x may work, it is deprecated so we strongly recommend you use Python 3 instead), as well as Scikit-Learn ≥0.20 and TensorFlow ≥2.0.

In [20]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Is this notebook running on Colab or Kaggle?
IS_COLAB = "google.colab" in sys.modules
IS_KAGGLE = "kaggle_secrets" in sys.modules

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

if not tf.config.list_physical_devices('GPU'):
    print("No GPU was detected. CNNs can be very slow without a GPU.")
    if IS_COLAB:
        print("Go to Runtime > Change runtime and select a GPU hardware accelerator.")
    if IS_KAGGLE:
        print("Go to Settings > Accelerator and select GPU.")

# Common imports
import numpy as np
import os
import pandas as pd

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "cnn"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

In [2]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator, load_img

IMAGE_WIDTH=224
IMAGE_HEIGHT=224
IMAGE_SIZE=(IMAGE_WIDTH, IMAGE_HEIGHT)
IMAGE_CHANNELS=1

batch_size=64


train_datagen = ImageDataGenerator(
    rotation_range=15,
    rescale=1./255,
    shear_range=0.1,
    zoom_range=0.2,
    width_shift_range=0.1,
    height_shift_range=0.1,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
        "../input/data-science-capstone-project-spring-2022/training_data/training_data",
    target_size=IMAGE_SIZE,
    batch_size=batch_size,
    class_mode='sparse',
    subset='training',
    color_mode='grayscale'
)

Found 27785 images belonging to 800 classes.


In [3]:
def get_hub_url_and_isize(model_name, ckpt_type, hub_type):
  if ckpt_type == '1k':
    ckpt_type = ''  # json doesn't support empty string
  else:
    ckpt_type = '-' + ckpt_type  # add '-' as prefix
  
  hub_url_map = {
    'efficientnetv2-b0': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0/{hub_type}',
    'efficientnetv2-b1': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1/{hub_type}',
    'efficientnetv2-b2': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2/{hub_type}',
    'efficientnetv2-b3': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3/{hub_type}',
    'efficientnetv2-s':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s/{hub_type}',
    'efficientnetv2-m':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m/{hub_type}',
    'efficientnetv2-l':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l/{hub_type}',

    'efficientnetv2-b0-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k/{hub_type}',
    'efficientnetv2-b1-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k/{hub_type}',
    'efficientnetv2-b2-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k/{hub_type}',
    'efficientnetv2-b3-21k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k/{hub_type}',
    'efficientnetv2-s-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k/{hub_type}',
    'efficientnetv2-m-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k/{hub_type}',
    'efficientnetv2-l-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k/{hub_type}',
    'efficientnetv2-xl-21k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k/{hub_type}',

    'efficientnetv2-b0-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b0-21k-ft1k/{hub_type}',
    'efficientnetv2-b1-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b1-21k-ft1k/{hub_type}',
    'efficientnetv2-b2-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b2-21k-ft1k/{hub_type}',
    'efficientnetv2-b3-21k-ft1k': f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-b3-21k-ft1k/{hub_type}',
    'efficientnetv2-s-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-s-21k-ft1k/{hub_type}',
    'efficientnetv2-m-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-m-21k-ft1k/{hub_type}',
    'efficientnetv2-l-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-l-21k-ft1k/{hub_type}',
    'efficientnetv2-xl-21k-ft1k':  f'gs://cloud-tpu-checkpoints/efficientnet/v2/hub/efficientnetv2-xl-21k-ft1k/{hub_type}',
      
    # efficientnetv1
    'efficientnet_b0': f'https://tfhub.dev/tensorflow/efficientnet/b0/{hub_type}/1',
    'efficientnet_b1': f'https://tfhub.dev/tensorflow/efficientnet/b1/{hub_type}/1',
    'efficientnet_b2': f'https://tfhub.dev/tensorflow/efficientnet/b2/{hub_type}/1',
    'efficientnet_b3': f'https://tfhub.dev/tensorflow/efficientnet/b3/{hub_type}/1',
    'efficientnet_b4': f'https://tfhub.dev/tensorflow/efficientnet/b4/{hub_type}/1',
    'efficientnet_b5': f'https://tfhub.dev/tensorflow/efficientnet/b5/{hub_type}/1',
    'efficientnet_b6': f'https://tfhub.dev/tensorflow/efficientnet/b6/{hub_type}/1',
    'efficientnet_b7': f'https://tfhub.dev/tensorflow/efficientnet/b7/{hub_type}/1',
  }
  
  image_size_map = {
    'efficientnetv2-b0': 224,
    'efficientnetv2-b1': 240,
    'efficientnetv2-b2': 260,
    'efficientnetv2-b3': 300,
    'efficientnetv2-s':  384,
    'efficientnetv2-m':  480,
    'efficientnetv2-l':  480,
    'efficientnetv2-xl':  512,
  
    'efficientnet_b0': 224,
    'efficientnet_b1': 240,
    'efficientnet_b2': 260,
    'efficientnet_b3': 300,
    'efficientnet_b4': 380,
    'efficientnet_b5': 456,
    'efficientnet_b6': 528,
    'efficientnet_b7': 600,
  }
  
  hub_url = hub_url_map.get(model_name + ckpt_type)
  image_size = image_size_map.get(model_name, 224)
  return hub_url, image_size

A couple utility functions to plot grayscale and RGB images:

In [4]:
def plot_image(image):
    plt.imshow(image, cmap="gray", interpolation="nearest")
    plt.axis("off")

def plot_color_image(image):
    plt.imshow(image, interpolation="nearest")
    plt.axis("off")

## Data preprocessing



In [5]:
from tensorflow.keras.utils import image_dataset_from_directory

train_dataset = image_dataset_from_directory(
    "../input/data-science-capstone-project-spring-2022/training_data/training_data",
    validation_split=0.2,
    subset="training",
    seed=1337,
    image_size=(50, 50),
    batch_size=32)
validation_dataset = image_dataset_from_directory(
    "../input/data-science-capstone-project-spring-2022/training_data/training_data",
    validation_split=0.2,
    seed=1337,
    subset="validation",
    image_size=(50, 50),
    batch_size=32)

Found 34428 files belonging to 800 classes.
Using 27543 files for training.


2022-04-27 01:34:33.578737: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 AVX512F FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-04-27 01:34:33.579178: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 01:34:33.579977: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2022-04-27 01:34:33.580607: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:937] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA 

Found 34428 files belonging to 800 classes.
Using 6885 files for validation.


Let’s look at the output of one of these Dataset objects: it yields batches of `180 × 180` RGB images (shape `(32, 180, 180, 3)`) and integer labels (shape `(32,)`). There are 32 samples in each batch (the batch size).

In [6]:
for data_batch, labels_batch in train_dataset:
    print("data batch shape:", data_batch.shape)
    print("labels batch shape:", labels_batch.shape)
    break

data batch shape: (32, 50, 50, 3)
labels batch shape: (32,)


2022-04-27 01:34:36.432991: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)


## Leveraging a pretrained model

In [7]:
import tensorflow_hub as hub
from tensorflow.keras import layers

model_name = 'efficientnetv2-b0' #@param {type:'string'}
ckpt_type = '1k'   # @param ['21k-ft1k', '1k']
hub_type = 'feature-vector' # @param ['classification', 'feature-vector']
hub_url, image_size = get_hub_url_and_isize(model_name, ckpt_type, hub_type)

tf.keras.backend.clear_session()

data_augmentation = keras.Sequential(
    [
      layers.Resizing(224, 224),
      layers.RandomRotation(0.1),
      layers.RandomTranslation(0, 0.1),
      layers.RandomTranslation(0.1, 0),
      layers.RandomZoom(0.1),
      layers.RandomCrop(200,200),
    ])

inputs = keras.Input(shape=(50, 50, 3))
x = data_augmentation(inputs)
#x = keras.layers.Rescaling(1./255)(x)
# Apply input value scaling.
x = hub.KerasLayer(hub_url, trainable=True)(x)
x = layers.Dropout(0.5)(x)
outputs = layers.Dense(800, activation='softmax')(x)
model = keras.Model(inputs, outputs)

2022-04-27 01:34:37.043422: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


In [8]:
model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 50, 50, 3)]       0         
_________________________________________________________________
sequential (Sequential)      (None, 200, 200, 3)       0         
_________________________________________________________________
keras_layer (KerasLayer)     (None, 1280)              5919312   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense (Dense)                (None, 800)               1024800   
Total params: 6,944,112
Trainable params: 6,883,504
Non-trainable params: 60,608
_________________________________________________________________


In [9]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.1, patience=5, verbose=1)

callbacksa = [
    keras.callbacks.ModelCheckpoint(
        filepath="ev2_fine_tuning_0420.keras",
        save_best_only=True,
        monitor="val_loss"), lr_scheduler
]

model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=0.1, momentum=0.9),
              metrics=["accuracy"])

history = model.fit(
    train_dataset,
    epochs=50,
    validation_data=validation_dataset,
    callbacks=callbacksa)

Epoch 1/50


2022-04-27 01:34:54.501505: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8005


861/861 [==============================] - 104s 108ms/step - loss: 4.0934 - accuracy: 0.2169 - val_loss: 1.6224 - val_accuracy: 0.5852
Epoch 2/50
861/861 [==============================] - 92s 107ms/step - loss: 1.1051 - accuracy: 0.7080 - val_loss: 0.4654 - val_accuracy: 0.8777
Epoch 3/50
861/861 [==============================] - 92s 107ms/step - loss: 0.5641 - accuracy: 0.8437 - val_loss: 0.3273 - val_accuracy: 0.9095
Epoch 4/50
861/861 [==============================] - 92s 107ms/step - loss: 0.3871 - accuracy: 0.8926 - val_loss: 0.2810 - val_accuracy: 0.9201
Epoch 5/50
861/861 [==============================] - 92s 107ms/step - loss: 0.2805 - accuracy: 0.9203 - val_loss: 0.2663 - val_accuracy: 0.9255
Epoch 6/50
861/861 [==============================] - 93s 108ms/step - loss: 0.2343 - accuracy: 0.9321 - val_loss: 0.2115 - val_accuracy: 0.9402
Epoch 7/50
861/861 [==============================] - 93s 107ms/step - loss: 0.1879 - accuracy: 0.9466 - val_loss: 0.2005 - val_accuracy: 0.

In [11]:
model.save("my_keras_model_0420.keras")

## Evaluation

In [14]:
!pip install --upgrade --no-cache-dir gdown -qq

In [15]:
# https://drive.google.com/file/d/1e2ydUonWb8YkyoSrfneJnC1NtuwHw2He/view?usp=sharing
!gdown --id 1e2ydUonWb8YkyoSrfneJnC1NtuwHw2He

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=1e2ydUonWb8YkyoSrfneJnC1NtuwHw2He
To: /kaggle/working/mid_test_0420.zip
100%|██████████████████████████████████████| 4.99M/4.99M [00:00<00:00, 44.3MB/s]


In [16]:
# https://drive.google.com/file/d/12Np4NVYdV_p0fZV6Csp2kxw6q4bV5pqL/view?usp=sharing
!gdown --id 12Np4NVYdV_p0fZV6Csp2kxw6q4bV5pqL

/opt/conda/lib/python3.7/site-packages/gdown/cli.py:131: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  category=FutureWarning,
Downloading...
From: https://drive.google.com/uc?id=12Np4NVYdV_p0fZV6Csp2kxw6q4bV5pqL
To: /kaggle/working/test_label_0420.txt
100%|██████████████████████████████████████| 56.2k/56.2k [00:00<00:00, 60.0MB/s]


In [17]:
!unzip -qq mid_test_0420.zip

In [18]:
!mkdir test_data
!mv all_test test_data/

In [21]:
df2 = pd.read_csv("test_label_0420.txt", header=None)
aaa = df2[0].tolist()
labels = [int(float(i)) for i in aaa]

In [22]:
test_dataset = image_dataset_from_directory(
    "/kaggle/working/test_data/",
    shuffle = False,
    image_size=(50, 50),
    batch_size=1)

Found 10000 files belonging to 1 classes.


In [23]:
prob = model.predict(test_dataset)

In [49]:
predictions = []
for i in range(0, prob.shape[0]):
    if np.max(prob[i,:]) < 0.99:
        predictions.append(-1)
    else:
        predictions.append(np.argmax(prob[i,:]))

In [50]:
from sklearn.metrics import f1_score
f1_score(labels, predictions, average='macro')

0.9395354261388255